In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the datetime module from the datetime library.
from datetime import datetime

# check do we have the config.py set up correctly and linked into this notebook? 
from config import weather_api_key

In [5]:
# Build the basic URL for the OpenWeatherMap with your weather_api_key added to the URL
# Also I already added the ViewJSON extension to my Chrome Browser

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create an endpoint URL for a city.
# make sure to reset the kernel and clear output before uploading this notebook or comment out the print statements
city_url = url + "&q=" + "cabo"

# Cabo and Cabo+San+Lucas are two different cities- Cabo alone brings up Cabo De Santo Augustinho in Brazil pop 208,000
# Remenber to remove or comment out the actual URL which includes the key so it is not displayed as open text
# print(city_url)

In [28]:
# Create a set of random latitude and longitude combinations.
# I think -90 to 90 is too likely to pick a non populated area
# Even using -60 to 70 you are going to get some extremely remote locations, most over water 
# adjust the max size to lower than 500 for the purpose of testing and increase to 1500 when ready
maxsize = 450
lats = np.random.uniform(low=-60.000, high=70.000, size=maxsize)
lngs = np.random.uniform(low=-180.000, high=180.000, size=maxsize)
lat_lngs = zip(lats, lngs)
lat_lngs

In [29]:
# this list() function does not seem to work.  Should it be preceded with pd.list() or some other class object? 
# no it just seems to be the size.  it doesn't like maxsize=500 or higher - it works when maxsize <= 450
newlats = list(lat_lngs)
# newlats
#for lat_lng in lat_lngs:
#    print(lat_lng[0], lat_lng[1])
len(newlats)

450

In [31]:
# Add the latitudes and longitudes to a list.
# coordinates = list(lat_lngs)
# len(coordinates)

# this fails because you only get to unzip ONCE
#so how about if I just assignthe newlats to a new variable name ? 
coordinates = newlats
len(coordinates)

450

In [32]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
cities

['port alfred',
 'mar del plata',
 'sao gabriel da cachoeira',
 'port lincoln',
 'isangel',
 'lazurne',
 'hambantota',
 'dong hoi',
 'palabuhanratu',
 'kpandu',
 'port hardy',
 'voh',
 'arraial do cabo',
 'simbahan',
 'asau',
 'bilibino',
 'carballo',
 'port blair',
 'thompson',
 'kokopo',
 'butaritari',
 'rikitea',
 'vaini',
 'birin',
 'evensk',
 'mahibadhoo',
 'ostersund',
 'avarua',
 'kapaa',
 'esperance',
 'taolanaro',
 'kelso',
 'chandvad',
 'severo-kurilsk',
 'coquimbo',
 'oxford',
 'ahuimanu',
 'santa maria',
 'ahipara',
 'nishihara',
 'saint george',
 'busselton',
 'sinfra',
 'tariba',
 'albany',
 'hithadhoo',
 'aswan',
 'awbari',
 'tasiilaq',
 'pechenga',
 'fredericksburg',
 'bengkulu',
 'bethanien',
 'vestmannaeyjar',
 'novyy svit',
 'salalah',
 'bambous virieux',
 'arcachon',
 'buncrana',
 'nome',
 'sistranda',
 'souillac',
 'pitimbu',
 'yerbogachen',
 'itacoatiara',
 'qasigiannguit',
 'wukari',
 'broome',
 'nikolskoye',
 'vicuna',
 'grand river south east',
 'klaksvik',
 'w

In [21]:
# okay so we have more than 500 cities but should we eliminate some with low populations? can we get population data? 
#I already know the nearest_city function only shows cities with population > 500 people
len(cities)

0

In [ ]:
# This is where the data import code begins in section 6.2.6

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")